In [1]:
#Import dependencies

from statistics import mean
from ipythonblocks import BlockGrid
from IPython.display import HTML, display, clear_output
from time import sleep
import numpy as np
import random
import copy
import collections
import numbers
from queue import PriorityQueue


#Set's some default information used by the Aima-Python Environments from the util.py f
orientations = EAST, NORTH, WEST, SOUTH = [(1, 0), (0, 1), (-1, 0), (0, -1)]
turns = LEFT, RIGHT = (+1, -1)


def turn_heading(heading, inc, headings=orientations):
    return headings[(headings.index(heading) + inc) % len(headings)]


def turn_right(heading):
    return turn_heading(heading, RIGHT)


def turn_left(heading):
    return turn_heading(heading, LEFT)


def distance(a, b):
    """The distance between two (x, y) points."""
    xA, yA = a
    xB, yB = b
    return np.hypot((xA - xB), (yA - yB))


def distance_squared(a, b):
    """The square of the distance between two (x, y) points."""
    xA, yA = a
    xB, yB = b
    return (xA - xB) ** 2 + (yA - yB) ** 2

#Direction class copied from agents.py from Aima-Python
class Direction:
    """A direction class for agents that want to move in a 2D plane
        Usage:
            d = Direction("down")
            To change directions:
            d = d + "right" or d = d + Direction.R #Both do the same thing
            Note that the argument to __add__ must be a string and not a Direction object.
            Also, it (the argument) can only be right or left."""

    R = "right"
    L = "left"
    U = "up"
    D = "down"

    def __init__(self, direction):
        self.direction = direction

    def __add__(self, heading):
        """
        >>> d = Direction('right')
        >>> l1 = d.__add__(Direction.L)
        >>> l2 = d.__add__(Direction.R)
        >>> l1.direction
        'up'
        >>> l2.direction
        'down'
        >>> d = Direction('down')
        >>> l1 = d.__add__('right')
        >>> l2 = d.__add__('left')
        >>> l1.direction == Direction.L
        True
        >>> l2.direction == Direction.R
        True
        """
        if self.direction == self.R:
            return {
                self.R: Direction(self.D),
                self.L: Direction(self.U),
            }.get(heading, None)
        elif self.direction == self.L:
            return {
                self.R: Direction(self.U),
                self.L: Direction(self.D),
            }.get(heading, None)
        elif self.direction == self.U:
            return {
                self.R: Direction(self.R),
                self.L: Direction(self.L),
            }.get(heading, None)
        elif self.direction == self.D:
            return {
                self.R: Direction(self.L),
                self.L: Direction(self.R),
            }.get(heading, None)


In [2]:
#Create basic thing class
class Thing():
    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))
    
    def is_alive(self):
        return hasattr(self, 'alive') and self.alive
    
    def state(self):
        print('Unkown state')
        
#Create a subclass Obstacle       
class Obstacle(Thing):
    pass

#Subclass of Obstacle used to set boundries of the environment
class Boundry(Obstacle):
    pass

#Create the person class from Obstacle to track a Person in the environment
class Person(Obstacle):
    pass

#Creates a Cleanable Thing that has 1 Attribute of Clean
class Cleanable(Thing):
    def __init__(self):
        self.clean = False

#Creates a Cleanable Thing Chair
class Chair(Cleanable):
    pass

#Creates a Cleanable Thing Trolley
class Trolley(Cleanable):
    pass


#Defines the Super class for Agent
class Agent(Thing):
    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        self.objects_cleaned=0
        if program is None:
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program
        

In [3]:
"""Environment used for Question 3"""

class UniHub():
    def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
        self.things = []
        self.agents = []
        self.width = width
        self.height = height
        self.observers = []
        # Sets iteration start and end (no walls).
        self.x_start, self.y_start = (0, 0)
        self.x_end, self.y_end = (self.width-1, self.height-1)
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        if display:
            self.grid.show()
            self.visible = True
        else:
            self.visible = False
        self.bounded = boundary
        self.colors = color
    
    def percept(self, agent):
        
        #Gets all things near the robot.
        return self.things_near(agent.location)
    
    perceptible_distance = 1

    def things_near(self, location, radius=None):
        """First it adds things within a radius of 1 (Up, down, left, and right).
            It then checks the corners around the robot and adds these to the list.
            Things in the same Cell as the robot are given a distance of 0.
            Things directly next to have a distance of 1, and things at the corners are 2.
            Returns list of all things surrounding the robot. And their distance"""
        if radius is None:
            radius = self.perceptible_distance
        radius2 = radius * radius
        thing_list = []
        for thing in self.things:
            #Add Directly next to items to list.
            if distance_squared(location, thing.location) <= radius2:
                thing_list.append((thing, distance_squared(location, thing.location)))
            
            #Checks each corner and adds them to the list
            elif thing.location[0]==location[0]-1 and thing.location[1]==location[1]+1:
                thing_list.append((thing, distance_squared(location, thing.location)))
            elif thing.location[0]==location[0]+1 and thing.location[1]==location[1]+1:
                thing_list.append((thing, distance_squared(location, thing.location)))
            elif thing.location[0]==location[0]+1 and thing.location[1]==location[1]-1:
                thing_list.append((thing, distance_squared(location, thing.location)))
            elif thing.location[0]==location[0]-1 and thing.location[1]==location[1]-1:
                thing_list.append((thing, distance_squared(location, thing.location)))
                
        return thing_list
    
    
    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing_super(self, thing, location=None):
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)
                
    def add_thing(self, thing, location=None, exclude_duplicate_class_items=False):
        """Add things to the world. If (exclude_duplicate_class_items) then the item won't be
        added if the location has at least one item of the same class."""
        if location is None:
            self.add_thing_super(thing)
        elif self.is_inbounds(location):
            if (exclude_duplicate_class_items and
                    any(isinstance(t, thing.__class__) for t in self.list_things_at(location))):
                return
            self.add_thing_super(thing, location)
        elif isinstance(thing, Boundry):
            if (exclude_duplicate_class_items and
                    any(isinstance(t, thing.__class__) for t in self.list_things_at(location))):
                return
            self.add_thing_super(thing, location)
                
    def is_inbounds(self, location):
        """Checks to make sure that the location is inbounds (within walls if we have walls)"""
        x, y = location
        return not (x < self.x_start or x > self.x_end or y < self.y_start or y > self.y_end)

    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end),
                    random.randint(self.y_start, self.y_end))
        if exclude is not None:
            while location == exclude:
                location = (random.randint(self.x_start, self.x_end),
                            random.randint(self.y_start, self.y_end))
        return location

    """Adds a Boundry on the outside of the grid."""
    def add_boundry(self):
        #Set a boundry around the area
        for x in range(self.width):
            self.add_thing(Boundry(), (x, -1))
            self.add_thing(Boundry(), (x, self.height))
        for y in range(0, self.height):
            self.add_thing(Boundry(), (-1, y))
            self.add_thing(Boundry(), (self.width, y))

    def turn_heading(self, heading, inc):
        """Return the heading to the left (inc=+1) or right (inc=-1) of heading."""
        return turn_heading(heading, inc)
    
    def step(self):
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
        else:
            print('{} Finished Cleaning All Chairs and Trolleys and has returned to state0'.format(str(agent)[1:-1]))

            
    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height
        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    def run(self, steps=1000, delay=1):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)
     
    def update(self, delay=1):
        sleep(delay)
        self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        # wait for the world to update and
        # apply changes to the same grid instead
        # of making a new one.
        clear_output(1)
        self.grid.show()
        self.visible = True

    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    
                    #Checks if the Instance is a cleanable object
                    if isinstance(world[x][y][-1], Cleanable):
                        
                        #If it is a cleanable object, check if clean and change color, else return default
                        if world[x][y][-1].clean:
                            if isinstance(world[x][y][-1], Chair):
                                self.grid[y, x] = (0, 200, 200)
                            else:
                                self.grid[y, x] = (65, 100, 115)
                        else:
                            self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]
                            
                    #Colors everything else normally
                    else:
                        self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))


    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        
        #Cleans an object that is dirty
        elif action == "clean":
            items = self.list_things_at(agent.location, tclass=Cleanable)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} Sprayed {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    items[0].clean=True
                    
    
    """Adds an object at a random location."""
    def add_thing_rand(self, thing):
        
        #Gets a random location of the environment
        loc=self.random_location_inbounds()
        
        #While something exists at the location specified, get a new random location.
        while self.list_things_at(loc) or loc==(0,0):
            loc=self.random_location_inbounds()
            
        #Once there is a blank location found, add it to the environment.
        self.add_thing(thing, loc)
        
    
    """A setup function that defines the environment locations of each object randomly"""
    def setup_environment(self, agent):
        
        #Sets up the boundry around the environment
        self.add_boundry()
        
        #Adds the agent to the start location
        self.add_thing(agent, [0,0])
        
        #Randomly places a person in the environment
        self.add_thing_rand(Person())
        
        #Gets a random number of cleanable objects where objects<width or m<n
        objects=random.randint(1,self.width-1)
        
        #Places each object randomly in the environment, randomly choosing a chair or trolley.
        for i in range(objects):
            choice=random.choice((1,2))
            if choice == 1:
                self.add_thing_rand(Chair())
            else:
                self.add_thing_rand(Trolley())

                
                    
    def is_done(self):
        """The agent is done once all objects are clean and the agent has returned to the starting point."""
        
        #Creates a list of dirty objects.
        dirty=[]
        for thing in self.things:
            if isinstance(thing, Cleanable):
                if not thing.clean:
                    dirty.append(thing)
        
        #Returns True if dirty list is empty (No dirty objects in the environment) and agent is home.
        return not dirty and agent.location==[0,0]


# Question 3.1 - Robot using only cameras

In [4]:
"""Agent class used for Question 3.1 and 3.2 """

class COVIDRobot(Agent):
    #Agents initial location and direction
    location = [0,0]
    direction = Direction("right")
    
    #Flag set to return home if object has been cleaned.
    return_home = False
    last_location = [0,0]
    search_dir = 'right'
    go_to_row = False
    current_row = 0
    next_row = 0
    return_to_location=False
    at_bottom = False
    
    x_boundry=None
    y_boundry=None
    person_at=None

    
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.performance+=1
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
            
    def turn(self, d):
        self.performance+=1
        self.direction = self.direction + d
        
    def clean(self, thing):
        if isinstance(thing,Cleanable):
            self.objects_cleaned+=1
            return True
        return False
        


In [5]:
def program(percepts):
    #Defaults to all potential options
    #Sets the starting location
    home=[0,0]
    
    #Sets a flag of areas that contain an obstacle relevent to the agent's location.
    blocked = {
        "left": False,
        "upleft": False,
        "downleft": False,
        "right": False,
        "upright":False,
        "downright": False,
        "up": False,
        "down": False}
    
    #Sets a flag if forward movement is possible. Defaults to True until percepts are checked.
    forward = True
    
    #Sets a flag if the agent detects a boundry
    boundry = {
        "left": False,
        "upleft": False,
        "downleft": False,
        "right": False,
        "upright":False,
        "downright": False,
        "up": False,
        "down": False}
    
    #Creates three lists based on distance. P1 for 1 move away, P2 for 2 moves, and p3 if 3 moves.
    p1=[]
    p2=[]
    p3=[]
    
    
    """A function to check where an obstacle is within the environment.
       First checks where the obstacle is, sets the blocked in the direction flag.
       Then it checks if that obstacle is also a boundry and sets the appropriate flag."""
    def check_obstacles(p):
        
        #Ensures the correct variables are setting the flags outside the function
        nonlocal boundry
        nonlocal blocked
        
        #Check if the obstacle is a person.  Agent will remember a person is at this location.
        if isinstance(p,Person):
            agent.person_at=p.location
            
        #Checks if an obstacle is to the right
        if p.location[0]==agent.location[0]+1:
            
            #Checks if the obstacle is up and right
            if p.location[1]==agent.location[1]-1:
                blocked["upright"]=True
                if isinstance(p,Boundry):
                    boundry["upright"]=True
                    
            #Checks if the obstacle is down and right
            elif p.location[1]==agent.location[1]+1:
                blocked["downright"]=True
                if isinstance(p,Boundry):
                    boundry["downright"]=True
                    
            #Else it is directly to the right
            else:        
                blocked["right"]=True
                if isinstance(p,Boundry):
                    boundry["right"]=True
                    agent.x_boundry=agent.location[0]
                    
        #Checks if an obstacle is to the left side of the environment
        elif p.location[0]==agent.location[0]-1:
            
            #Checks if the obstacle is up and left
            if p.location[1]==agent.location[1]-1:
                blocked["upleft"]=True
                if isinstance(p,Boundry):
                    boundry["upleft"]=True
                    
            #Checks if the obstacle is down and left
            elif p.location[1]==agent.location[1]+1:
                blocked["downleft"]=True
                if isinstance(p,Boundry):
                    boundry["downleft"]=True
            
            #Else obstacle is just left.
            else:        
                blocked["left"]=True
                if isinstance(p,Boundry):
                    boundry["left"]=True
                    
        #Last checks if the obstacle is directly up or down.
        elif p.location[1]==agent.location[1]+1:
            blocked["down"]=True
            if isinstance(p,Boundry):
                boundry["down"]=True
                agent.y_boundry=agent.location[1]
        elif p.location[1]==agent.location[1]-1:
            blocked["up"]=True
            if isinstance(p,Boundry):
                boundry["up"]=True
                
    """Checks if forward movemnt is possible based on where obstacles have been detected."""
    def check_forward():
        nonlocal forward
        if agent.direction.direction==Direction.R and blocked["right"]:
            forward=False
        elif agent.direction.direction==Direction.L and blocked["left"]:
            forward=False
        elif agent.direction.direction==Direction.U and blocked["up"]:
            forward=False
        elif agent.direction.direction==Direction.D and blocked["down"]:
            forward=False
            
    """Checks if the object directly in front of the agent is cleanable and is dirty."""        
    def check_forward_clean(p):
        if agent.direction.direction == Direction.R and agent.location[0]+1==p.location[0]:
            if isinstance(p, Cleanable) and p.clean==False:
                return True
        elif agent.direction.direction == Direction.L and agent.location[0]-1==p.location[0]:
            if isinstance(p, Cleanable) and p.clean==False:
                return True
        elif agent.direction.direction == Direction.D and agent.location[1]+1==p.location[1]:
            if isinstance(p, Cleanable) and p.clean==False:
                return True
        elif agent.direction.direction == Direction.U and agent.location[1]-1==p.location[1]:
            if isinstance(p, Cleanable) and p.clean==False:
                return True
        else:
            return False
        
    
    """Function that gets the agent back to the starting location"""    
    def return_home():
        
        """Checks if the Agent is already at starting location.
            If the agent has reached home, set return_home to false and begin to return_to_location."""
        if agent.location==[0,0]:
            agent.return_home=False
            agent.return_to_location=True
            
            #Returns the best turn option based on direction. Left, turns down (left). Up returns Turn right.
            if agent.direction.direction == Direction.L:
                return 'turnleft'
            else:
                return 'turnright'
        else:
            
            #Check if the agent is facing left.
            if agent.direction.direction == Direction.L:
                
                #If the agent is further away then x=1, and can move forward, go forward.
                if forward and agent.location[0]>1:
                    return 'moveforward'
                
                #If the agent is at x=1 or x=0 check before moving forward.
                #If the agent sees and obstacle up and forward, turns right (up).
                #This prevents the agent from getting stuck between boundry and object above.
                #More efficient then hitting boundry and turning around.
                elif forward and agent.location[0]<=1:
                    
                    #First, if the agent is at the top and can move forward go forward.
                    if agent.location[1]==0 or not blocked['upleft']:
                        return 'moveforward'
                       
                    elif blocked['upleft']:
                        return 'turnright'
                
                #Finally if the agent cannot move forward and the boundry is above the agent, go down, else turn right.
                elif boundry['up']:
                    return 'turnleft'
                else:
                    return 'turnright'


            #Check if facing Up
            elif agent.direction.direction == Direction.U:
                
                #If the agent is greater then y=1, and can go forward, go forward
                if forward and agent.location[1]>1:
                    return 'moveforward'
                
                #If the agent is one cell away or at the top the agent needs to ensure the agent doesn't get stuck.
                #Checks if the agent is at the left of the environment, if so movefoward to [0,0]
                #Checks if the agent can see a block up and forward that will have the agent get stuck, turn left to avoid.
                elif forward and agent.location[1]<=1:
                    if agent.location[0]==0:
                        return 'moveforward'
                    
                    #Ensures the agent can still go forward if not at the top.
                    #This obstacle would exist at [1,1] meaning the agent can safely go around without getting stuck.
                    elif not blocked['upleft']:
                        return 'moveforward'
                    
                    elif blocked['upleft']:
                        return 'turnleft'
                    
                #Finally if at left boundry and can't go forward, turn right, else turn left.
                elif boundry['left']:
                    return 'turnright'
                else:
                    return 'turnleft'

            #The Agent should only be facing Right if avoiding an obstacle or if it faced that direction
            #When cleaning.
            elif agent.direction.direction==Direction.R:
                
                #If there is something above the agent and it's not the upper boundry 
                #moveforward to go around obstacle, else turn around
                if blocked["up"] and not boundry['up'] and blocked['left'] and forward:
                    return 'moveforward'
                else:
                    return 'turnleft'
                
            #The Agent should only be facing Down if avoiding an obstacle or if it faced that direction
            #When cleaning. 
            elif agent.direction.direction==Direction.D:
                
                #If the agent is blocked to the left and not at the top, go forward to avoid the obstacle.
                if blocked["left"] and boundry['up'] and forward:
                    return 'moveforward'
                else:
                    return 'turnright'

    """A function to get the agent back to where it was before returning home.  As the agent may not have a clear
        understanding of exactly where the right and bottom boundries exist at all times, this bases direction
        on relevant agent location to destination location."""
    def return_to_location():
        
        #Checks if the agent has already reached the last location.
        if agent.location==agent.last_location:
            agent.return_to_location=False
            return next_move()                
            
        else:
            if agent.direction.direction == Direction.R:
                
                #If the agent is to the left of destination, moveforward
                #Can't really prevent hitting a corner and turning around without knowing the X boundry at start
                #Once the agent knows the boundry, this is stored.
                if agent.location[0]<agent.last_location[0] and forward:
                    
                    #If the boundry is known, check if the location is in the corner, else avoid.
                    if agent.x_boundry:
                        if agent.location[0]+1==agent.x_boundry:
                            if agent.location[1]==agent.last_location[1] or not blocked['downright']:
                                return 'moveforward'
                            elif blocked['downright']:
                                return 'turnright'
                        else:
                            return 'moveforward'
                    else:
                        return 'moveforward'
                
                #Decides what to do if the agent is at or to the right of the location or cannot move forward.
                elif agent.location[0]>=agent.last_location[0] or not forward:
                    if blocked['down'] and forward:
                        return 'moveforward'
                    elif boundry['down'] and not forward:
                        return 'turnleft'
                    #Checks if the agent is near the upper boundry
                    elif agent.location[1]<=agent.last_location[1]:
                        
                        #This ensures the agent 
                        if blocked["downright"] and not boundry["up"]:
                            return 'turnright'
                        elif blocked['down'] and boundry['left']:
                            return 'moveforward'
                        else:
                            return 'turnright'
                    elif agent.location[1]>agent.last_location[1]:
                        return 'turnleft'

            #Check if the agent is facing down.  Same issue as right, need to check if going into corner.        
            elif agent.direction.direction == Direction.D:
                if agent.location[1]<agent.last_location[1] and forward:
                    
                    #Check if bottom boundry is known. If not known move forward, else avoid obstacles
                    if agent.y_boundry:
                        if agent.location[1]+1==agent.y_boundry:
                            if agent.location[0]==agent.last_location[0] or not blocked['downright']:
                                return 'moveforward'
                            elif blocked['downright']:
                                return 'turnleft'
                        else:
                            return 'moveforward'
                    else:
                        return 'moveforward'
                
                #Checks if the agent is below or at the destination Y
                elif agent.location[1]>=agent.last_location[1] or not forward:
                    
                    #If the agent is blocked on the right move forward and go around.
                    if blocked['right'] and forward:
                        return 'moveforward'
                    
                    #If the agent is to the right of the destination turn to go left.
                    elif agent.location[0]>agent.last_location[0]:
                        return 'turnright'
                    
                    #else as long as the agent isn't in a corner at the boundry, turn to go right.
                    elif agent.location[0]<=agent.last_location[0]:
                        if blocked['down'] and boundry['right']:
                            return 'turnright'
                        else:
                            return 'turnleft'
                    
            #Checks if the Agent is left or right of the destination and moves accordingly        
            elif agent.direction.direction == Direction.L:
                if agent.location[0]>agent.last_location[0] and forward:
                    return 'moveforward'
                elif agent.location[0]<=agent.last_location[0] or not forward:
                    if agent.location[1]>agent.last_location[1]:
                        return 'turnright'
                    elif agent.location[1]<=agent.last_location[1]:
                        if forward and blocked['down'] and boundry['right']:
                            return 'moveforward'
                        else:
                            return 'turnleft'
                        
            #Checks if the agent should move up or turn based on relevant location to destination.         
            elif agent.direction.direction == Direction.U:
                if boundry['down'] and blocked['right'] and forward:
                    return 'moveforward'
                elif agent.location[1]>agent.last_location[1] and forward:
                    return 'moveforward'
                elif agent.location[1]<=agent.last_location[1] or not forward:
                    if agent.location[0]<agent.last_location[0]:
                        return 'turnright'
                    elif agent.location[0]>=agent.last_location[0]:
                        return 'turnleft'
    
    """Function to check the percepts the robots sees and classify."""
    def check_percepts():
        nonlocal p1,p2,p3
        for p in percepts:
            
            #If the percept is an obstacle, check where it is.
            if isinstance(p[0],Obstacle):
                check_obstacles(p[0])
            
            #Else check distance to agent.  If 0, check if it needs to be cleaned
            elif p[1]==0 and not agent.return_home:
                if isinstance(p[0], Cleanable) and p[0].clean==False:
                    
                    #Set the return to home flag.
                    agent.return_home=True
                    
                    #If the agent was returning to a previous location when it found object, don't update last_location
                    #This is to ensure everywhere is searched.
                    if not agent.return_to_location:
                        agent.last_location=agent.location.copy()
                        
                        
                        #If the agent cleans an object at a boundry, set the location to be the next row to search.
                        if agent.search_dir=='right' and agent.location[1]==agent.current_row and boundry['right']:
                            agent.last_location[0]=0
                            agent.next_row=agent.current_row+3
                            agent.go_to_row=True
                        elif agent.search_dir=='left' and agent.location[1]==agent.current_row and boundry['left']:
                            if agent.x_boundry:
                                agent.last_location[0]=agent.x_boundry
                                agent.next_row=agent.current_row+3
                                agent.go_to_row=True
                        
                        #Ensures that the agent doesn't store the last_location as the same location as a person.
                        if agent.person_at:
                            if agent.person_at[0] == agent.last_location[0] and agent.person_at[1] == agent.current_row:
                                pass
                            else:
                                agent.last_location[1]=agent.current_row
                        else:
                            agent.last_location[1]=agent.current_row
                                
            
                    return 'clean'

            #Adds the Percept to a list according to distance.  Only adds objects that require cleaning.
            elif p[1]==1:
                if isinstance(p[0], Cleanable) and p[0].clean==False:
                    p1.append(p[0])
            
            elif p[1]==2:
                if isinstance(p[0], Cleanable) and p[0].clean==False:
                    p2.append(p[0])
            else:
                p3.append(p)
        
        #Runs the check_forward function
        check_forward()
                    
    """Function that moves the agent towards a cleanable object."""
    def move_towards_object():
        
        #if percept is next to agent check if front of the agent 
        for p in p1:
            if forward and check_forward_clean(p):
                return 'moveforward'
            
            #Else Turn in the correct direction.
            #If the object is actually behind the agent, move to the p3 list.
            else:   
                if agent.direction.direction == Direction.R:
                    if agent.location[1]+1==p.location[1]:
                        return 'turnright'
                    elif agent.location[1]-1==p.location[1]:
                        return 'turnleft'
                    else:
                        p3.append(p)
                elif agent.direction.direction == Direction.L:
                    if agent.location[1]-1==p.location[1]:
                        return 'turnright'
                    elif agent.location[1]+1==p.location[1]:
                        return 'turnleft'
                    else:
                        p3.append(p)
                elif agent.direction.direction == Direction.U:
                    if agent.location[0]+1==p.location[0]:
                        return 'turnright'
                    elif agent.location[0]-1==p.location[0]:
                        return 'turnleft'
                    else:
                        p3.append(p)
                elif agent.direction.direction == Direction.D:
                    if agent.location[0]-1==p.location[0]:
                        return 'turnright'
                    elif agent.location[0]+1==p.location[0]:
                        return 'turnleft'
                    else:
                        p3.append(p)

        #If no objects next to the agent, check the corner objects (distance 2)
        #If the object is in the direction the agent is facing and it is not blocked, it will move forward.
        #Else it will turn left or right based on where the object is or append to p3 list if behind agent.
        for p in p2:
            
            if agent.direction.direction == Direction.R:
                if agent.location[1]+1==p.location[1] and agent.location[0]+1==p.location[0]:
                    if forward:
                        return 'moveforward'
                    else:
                        return 'turnright'
                elif agent.location[1]-1==p.location[1] and agent.location [0]+1==p.location[0]:
                    if forward:
                        return 'moveforward'
                    else:
                        return 'turnleft'
                else:
                    p3.append(p)
            
            elif agent.direction.direction == Direction.L:
                if agent.location[1]-1==p.location[1] and agent.location[0]-1==p.location[0]:
                    if forward:
                        return 'moveforward'
                    else:
                        return 'turnright'
                elif agent.location[1]+1==p.location[1] and agent.location[0]-1==p.location[0]:
                    if forward:
                        return 'moveforward'
                    else:
                        return 'turnleft'
                else:
                    p3.append(p)
            elif agent.direction.direction == Direction.U:
                if agent.location[0]+1==p.location[0] and agent.location[1]-1==p.location[1]:
                    if forward:
                        return 'moveforward'
                    else:

                        return 'turnright'
                elif agent.location[0]-1==p.location[0] and agent.location[1]-1==p.location[1]:
                    if forward:
                        return 'moveforward'
                    else:
                        return 'turnleft'
                else:
                    p3.append(p)

            elif agent.direction.direction == Direction.D:
                if agent.location[0]-1==p.location[0] and agent.location[1]+1==p.location[1]:
                    if forward:
                        return 'moveforward'
                    else:

                        return 'turnleft'
                elif agent.location[0]+1==p.location[0] and agent.location[1]+1==p.location[1]:
                    if forward:
                        return 'moveforward'
                    else:

                        return 'turnright'
                else:
                    p3.append(p)
        
        #If only objects behind the agent, agent needs to turn around. Defaults to right.
        if p3:
            return 'turnright'
 

    """Function to determine the next move the agent should make.
        Order is:
            1. Check if returning home.
            2. If not going home, check if dirty object is within range.
            3. If no dirty object is within range, check if the agent is returning to a previous location
                *By having this after the check dirty object section, The agent can clean while returning.
                *In the check_percepts function, if the agent was already returning_to_location, it does not
                *overwrite the last_location attribute.
            4. Finally, if nothing else, continue searching."""
    def next_move():
        
        #Skips all other options if the agent is returning home. Prevents cleaning on the way.
        if agent.return_home:
            return return_home()
        
        #Next it checks if there are any cleanable objects within its sight.
        elif p1 or p2 or p3:
            return move_towards_object()
        
        #Then if nothing to clean, checks if it is returning to a location.
        elif agent.return_to_location:
            return return_to_location()
        
        #If none of the above, the agent defaults to a Left-to-Right, Right-to-Left search method.
        else:
            #Checks if the agent hit bottom while moving to the next row.
            if boundry["down"] and agent.next_row:
                agent.current_row=agent.location[1]
                agent.at_bottom=True
                
            #Checks if the agent has made it to the next row.
            elif agent.location[1]==agent.next_row and not agent.at_bottom:
                
                #Turns off the flag telling the program to move to the next search row.
                agent.go_to_row=False
                
                #Sets the current search row to agents current row.
                agent.current_row=agent.location[1]
                
                #Checks that the agent has actually hit the edge of the hub and sets the next search row.
                if (agent.search_dir=='right' and boundry['right']) or (agent.search_dir=='left' and boundry['left']):
                    
                    #As the agent can see one row above and one row below, it moves down 3 rows to maximize our search pattern
                    agent.next_row=agent.current_row+3
            
            #Checks if the agent is at the current searching row and moves accordingly.
            if agent.location[1]==agent.current_row:
                if agent.direction.direction == Direction.R:
                    if forward and not agent.go_to_row:
                        return 'moveforward'
                    
                    #Ensures the agent can avoid an obstacle(person) while going to the next row.
                    elif forward and agent.go_to_row and blocked["down"]:
                        return 'moveforward'
                    elif agent.go_to_row:
                        return 'turnright'
                    
                    #If the agent hits a boundry, set the next row flag and update the search direction
                    elif boundry["right"]:
                        agent.go_to_row=True
                        agent.search_dir='left'
                        return 'turnright'
                    
                    #Ensures that the agent does not get stuck in the bottom right corner.
                    elif boundry["left"] and boundry['down'] and blocked['right']:
                        return 'turnleft'
                    else:
                        return 'turnright'
                    
                    
                elif agent.direction.direction == Direction.L:
                    if forward and not agent.go_to_row:
                        return 'moveforward'
                    
                    #Ensures that the agent can go around an obstacle while changing rows.
                    elif forward and agent.go_to_row and blocked["down"]:
                        return 'moveforward'
                    elif agent.go_to_row:
                        return 'turnleft'
                    
                    #Ensures the agent doesn't get stuck if it's on the top row travelling left.
                    elif boundry["up"]:
                        return 'turnleft'
                    
                    #Checks if it hit the boundry while searching and set's agent.go_to_row flag.
                    elif boundry["left"]:
                        agent.go_to_row=True
                        agent.search_dir='right'
                        return 'turnleft'
                    
                    #Ensures the agent doesn't get stuck in the bottom left corner.
                    elif boundry['right'] and boundry['down'] and blocked['left']:
                        return 'turnright'
                    else:
                        return 'turnleft'
                    
                
                elif agent.direction.direction == Direction.D:
                    if agent.search_dir=='right':
                        if blocked["right"] and forward:
                            return 'moveforward'
                        elif forward and agent.go_to_row:
                            return 'moveforward'
                        else:
                            return 'turnleft'
                    
                    if agent.search_dir=='left':
                        if blocked["left"] and forward:
                            return 'moveforward'
                        elif forward and agent.go_to_row:
                            return 'moveforward'
                        else:
                            return 'turnright'
                    
                elif agent.direction.direction == Direction.U:
                    if blocked['left'] and boundry['down'] and blocked['right']:
                        return 'moveforward'
                    elif agent.search_dir=='right':
                        return 'turnright'
                    elif agent.search_dir=='left':
                        return 'turnleft'

            
            #If the agent is above the next row move accordingly.
            if agent.location[1] > agent.next_row:
                if agent.direction.direction == Direction.R:
                    if (blocked["upright"] or blocked["up"]) and forward:
                        return 'moveforward'
                    elif forward and not blocked["up"]:
                        return 'turnleft'
                    else:
                        return 'turnright'
                elif agent.direction.direction ==Direction.L:
                    if (blocked["upleft"] or blocked["up"]) and forward:
                        return 'moveforward'
                    elif forward and not blocked["up"]:
                        return 'turnright'
                    else:
                        return 'turnleft'
                elif agent.direction.direction == Direction.D:
                    if (blocked["right"] or blocked["left"]):
                        agent.current_row=agent.location[1]
                        if forward:
                            agent.go_to_row=False
                            agent.current_row=agent.location[1]+1
                            return 'moveforward'
                        elif boundry["left"]:
                            return 'turnleft'
                        else:
                            return 'turnright'
                    elif agent.search_dir=='right':
                        return 'turnleft'
                    elif agent.search_dir=='left':
                        return 'turnright'
                elif agent.direction.direction == Direction.U:
                    return 'moveforward'
            
            
            #If the agent is below the next_row move accordingly.
            if agent.location[1] < agent.next_row:
                if agent.direction.direction == Direction.R:
                    if (blocked["downright"]or blocked["down"]) and forward:
                        return 'moveforward'
                    elif forward and not blocked["down"]:
                        return 'turnright'
                    else:
                        return 'turnleft'
                
                if agent.direction.direction == Direction.L:
                    if (blocked["downleft"] or blocked["down"]) and forward:
                        return 'moveforward'
                    elif forward and not blocked["down"]:
                        return 'turnleft'
                    else:
                        return 'turnright'
                
                if agent.direction.direction == Direction.D:
                    if forward:
                        return 'moveforward'
                    elif agent.search_dir=='right':
                        return 'turnleft'
                    elif agent.search_dir=='left':
                        return 'turnright'
                
                if agent.direction.direction == Direction.U:
                    if not blocked['right']:
                        if agent.search_dir=='right':
                            return 'turnright'
                    if not blocked['left']:
                        if agent.search_dir=='left':
                            return 'turnleft'

    clean=check_percepts()
    #If check_percepts has returned a value, use it.
    if clean:
        return clean
    return next_move()


In [6]:
hub = UniHub(6,6, color={'COVIDRobot': (0,150,50), 'Trolley': (150,100,0), 'Chair': (100, 75, 0), 'Person': (250,0,0), 'Animal': (12,32,36)}) # park width is set to 5, and height to 20
agent = COVIDRobot(program)
hub.setup_environment(agent)

hub.run()

print("The COVIDRobot moved %d times and cleaned %d objects.  The overall performance (moves/objects) = %5.1f" % (agent.performance, agent.objects_cleaned, agent.performance/agent.objects_cleaned))

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


The COVIDRobot moved 62 times and cleaned 3 objects.  The overall performance (moves/objects) =  20.7


# Question 3.2 - Robot using a Search Algorithm (Returning to State 0)

In [22]:
class Uni_Hub_Algorithm(UniHub):
    def __init__(self, size=5, color={}):
        super().__init__(size,size,color)
        self.colors=color
        self.size = size
        #The Environment creates a State List and Dictionary (Relating States to locations)
        self.all_states, self.all_dict = self.create_states()
        self.dirty_objects = []
        
        #Sets up the Environment by adding objects
        self.setup_environment()
        
        #The environment finds all possible connections
        self.connections=self.get_connections()
        
        #Creates a Dictionary ready to send connections to the AgentProgram
        self.percepts={
            'connections': self.connections,
            'state_dict': self.all_dict
            
        }
        
    def percept(self,agent):
        #Adds nearby objects to the percept dictionary
        self.percepts['things_near']=self.things_near(agent.location)
        
        #Adds the remaining Dirty objects to ensure all get cleaned
        self.percepts['dirty_remaining']=self.dirty_objects
        
        return self.percepts
    
    """Creates all available state locations"""
    def create_states(self):
        all_states=[]
        all_dict={}
        count=0
        
        #Creates a list of all possible state locations
        for i in range(0,self.size):
            for j in range(0,self.size):
                all_states.append([i,j])
                
        #Creates a dictionary to find state locations by state name
        for state in all_states:
            all_dict['state%d' % (count)]= state
            count+=1
            
        return all_states, all_dict
    
    """Returns all possible adjacent locations based on a specific location"""
    def get_adjacent_states(self, current):
        adjacent_states=[]
        neighbours = [[-1,0],[1,0],[0,1],[0,-1]]
        for state in neighbours:
            if self.all_states.count([current[0]+state[0],current[1]+state[1]])>0:
                adjacent_states.append([current[0]+state[0],current[1]+state[1]])
        return current,adjacent_states
    
    """Returns all possible states and their connections as a Dictionary of Dictionaries"""
    def get_connections(self):
        connections={}
        
        for state in self.all_states:
            root, neighbours = self.get_adjacent_states(state)
            adj={}
            
            for s in neighbours:
                adj['state{}'.format(self.all_states.index(s))] = 1
            connections['state{}'.format(self.all_states.index(state))] = adj
        
        return connections
            
    """The Environment generates a list of all the dirty object locations"""
    def get_dirty_objects(self):
        dirty=[]
        for thing in self.things:
            if isinstance(thing, Cleanable):
                if not thing.clean:
                    dirty.append(self.all_dict['state%d' % (self.all_states.index([thing.location[0], thing.location[1]]))])
        return dirty

    """Generates the Random Environment variables"""            
    def setup_environment(self):        
        #Randomly places a person in the environment
        people=random.randint(1,int(self.width/2))
        for i in range(people):
            self.add_thing_rand(Person())
        
        objects=random.randint(1,self.width-1)
        
        #Places each object randomly in the environment, randomly choosing a chair or trolley.
        for i in range(objects):
            choice=random.choice((1,2))
            if choice == 1:
                self.add_thing_rand(Chair())
            else:
                self.add_thing_rand(Trolley())
        
        self.dirty_objects=self.get_dirty_objects()
        
        
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'moveright':
            print('{} decided to move {} at location: {}'.format(str(agent)[1:-1], 'right', agent.location))
            agent.move('right')
        elif action == 'moveleft':
            print('{} decided to move {} at location: {}'.format(str(agent)[1:-1], 'left', agent.location))
            agent.move('left')
        elif action == 'moveup':
            print('{} decided to move {} at location: {}'.format(str(agent)[1:-1], 'up', agent.location))
            agent.move('up')
        elif action == 'movedown':
            print('{} decided to move {} at location: {}'.format(str(agent)[1:-1], 'down', agent.location))
            agent.move('down')
        
        #Cleans an object that is dirty
        elif action == "clean":
            items = self.list_things_at(agent.location, tclass=Cleanable)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} Sprayed {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    items[0].clean=True
                    self.dirty_objects.remove([items[0].location[0],items[0].location[1]])
                    
                    
    

In [23]:
"""Covid Robot agent for Task 3.2 and 3.3"""
class COVIDRobot(Agent):
    def __init__(self, program=None, goal=None ,initial_state='state0'):
        super().__init__(program)
        self.state = initial_state
        self.seq = []
        self.goal = goal

    def move(self, direction):
        self.performance+=1
        if direction=='right':
            self.location[0]+=1
            
        elif direction=='left':
            self.location[0]-=1
            
        elif direction=='up':
            self.location[1]-=1
        elif direction=='down':
            self.location[1]+=1
    
    def clean(self, thing):
        if isinstance(thing,Cleanable):
            self.objects_cleaned+=1
            return True
        return False
    


In [24]:
def program(percepts):
    
    #Sets an empty list to store all objects found at agents location
    p0=[]
    
    """Functions used for A* search to find the best available path."""
    
    """Reconstructs the path found from the search, finds the next state and appends to the path list"""
    def reconstruct_path(came_from, start, goal):

        current = goal
        path = []
        while current != start:
            path.append(current)
            current = came_from[current]
        path.reverse()
        return path

    """Basic Heuristic of distance"""
    def heuristic(current, goal):
        x1,y1=current
        x2,y2=goal
        return abs(x1-x2)+abs(y1-y2)
    
    """A* search Algorithm"""
    def search(connections, start=agent.state, goal=agent.goal, state_dict=percepts['state_dict']):
        start=start
        goal=goal
        state_dict=state_dict
        frontier = PriorityQueue()
        frontier.put((0,start))
        came_from = {}
        cost_so_far = {}
        came_from[start] = None
        cost_so_far[start] = 0
        
        while not frontier.empty():
            current = frontier.get()[1]
            
            if current == goal:
                break

            for i in connections[current]:
                new_cost = cost_so_far[current] + 1
                if i not in cost_so_far or new_cost < cost_so_far[i]:
                    cost_so_far[i] = new_cost
                    priority = new_cost + heuristic(state_dict[i], state_dict[goal])
                    frontier.put((priority,i))
                    came_from[i] = current
                    
        return reconstruct_path(came_from, start, goal)
    
    """Checks percepts for cleanable objects at location and adds them to a list"""
    for p in percepts['things_near']:
        if p[1]==0 and isinstance (p[0], Cleanable):
            p0.append(p[0])
            
        #Checks if the percept is a person.
        elif isinstance(p[0], Person):
            
            #If the persons state is in the current move sequence, the program generates a new connection list
            #removing the persons state.  It then regenerates a new seq avoiding the person.
            loc=[p[0].location[0],p[0].location[1]]
            state=list(percepts['state_dict'].keys())[list(percepts['state_dict'].values()).index(loc)]
            
            #Removes person state from connections
            if agent.seq and state in agent.seq:
                for q,v in percepts['connections'].items():
                    if state in v:
                        del v[state]
                        
                #Resets agent sequence
                agent.seq =[]
                
                #Generates new sequence
                agent.seq = new_best_path=search(percepts['connections'], agent.state, agent.goal)
    
    #If the agent has a goal and the goal is not state0
    if agent.goal and agent.goal!='state0':
        
        #Check if the Agent is at the goal state and if it can see the dirty object
        if agent.state==agent.goal and agent.goal!='state0':
            for i in p0:
                if isinstance(i, Cleanable) and i.clean==False:
                    agent.goal='state0'
                    return 'clean'
        else:
            next_state=agent.seq.pop(0)
                
    #If the goal state is to return and the agent is not home
    elif agent.goal=='state0' and agent.state!=agent.goal:
        if not agent.seq:
            agent.seq=search(percepts['connections'], agent.state, 'state0')
        
        next_state=agent.seq.pop(0)

    #If the agent is at the goal or there is no goal, the program searchs for the closest available dirty object.
    elif agent.state==agent.goal or not agent.goal:
        
        #Iterate through each object and choose the shortest path to an object.
        for i in percepts['dirty_remaining']:
            j=list(percepts['state_dict'].keys())[list(percepts['state_dict'].values()).index(i)]
            if 'best_path' in locals():
                new_path=search(percepts['connections'], agent.state, j)
                if len(new_path)<len(best_path):
                    best_path=new_path.copy()
            else:
                best_path=search(percepts['connections'], agent.state, j)
        
        #Once beth path is determined, copy path to agent sequence
        agent.seq=best_path.copy()
        
        #Sets the agent's goal to the last state
        agent.goal=best_path[-1]
        
        next_state=agent.seq.pop(0)       
        
    #Use the state dictionary to get the actual Location of the state
    next_move=percepts['state_dict'][next_state]
    agent.state=next_state
    
    if agent.location[0]+1==next_move[0]:
        return 'moveright'
    elif agent.location[0]-1 == next_move[0]:
        return 'moveleft'
    elif agent.location[1]+1== next_move[1]:
        return 'movedown'
    elif agent.location[1]-1==next_move[1]:
        return 'moveup'         
    

In [26]:
UniHub_3_2=Uni_Hub_Algorithm(7, {'COVIDRobot': (0,150,50), 'Trolley': (150,100,0), 'Chair': (100, 75, 0), 'Person': (250,0,0)})
agent=COVIDRobot(program)
UniHub_3_2.add_thing(agent, [0,0])
UniHub_3_2.run()
print("The COVIDRobot moved %d times and cleaned %d objects.  The overall performance (moves/objects) = %5.1f" % (agent.performance, agent.objects_cleaned, agent.performance/agent.objects_cleaned))

,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


COVIDRobot decided to move down at location: [2, 0]


KeyboardInterrupt: 

# Question 3.3 - Robot cleans all objects before return to state 0

In [27]:
def program(percepts):
    #Sets an empty list to store all objects found at agents location
    p0=[]
    
    """Functions used for A* search to find the best available path."""
    
    """Reconstructs the path found from the search, finds the next state and appends to the path list"""
    def reconstruct_path(came_from, start, goal):

        current = goal
        path = []
        while current != start:
            path.append(current)
            current = came_from[current]
        path.reverse()
        return path

    """Basic Heuristic of distance"""
    def heuristic(current, goal):
        x1,y1=current
        x2,y2=goal
        return abs(x1-x2)+abs(y1-y2)
    
    """A* search Algorithm"""
    def search(connections, start=agent.state, goal=agent.goal, state_dict=percepts['state_dict']):
        start=start
        goal=goal
        state_dict=state_dict
        frontier = PriorityQueue()
        frontier.put((0,start))
        came_from = {}
        cost_so_far = {}
        came_from[start] = None
        cost_so_far[start] = 0
        
        while not frontier.empty():
            current = frontier.get()[1]
            
            if current == goal:
                break

            for i in connections[current]:
                new_cost = cost_so_far[current] + 1
                if i not in cost_so_far or new_cost < cost_so_far[i]:
                    cost_so_far[i] = new_cost
                    priority = new_cost + heuristic(state_dict[i], state_dict[goal])
                    frontier.put((priority,i))
                    came_from[i] = current
                    
        return reconstruct_path(came_from, start, goal)
    
    """Checks percepts for cleanable objects at location and adds them to a list"""
    for p in percepts['things_near']:
        if p[1]==0 and isinstance (p[0], Cleanable):
            p0.append(p[0])
            
        #Checks if the percept is a person.
        elif isinstance(p[0], Person):
            
            #If the persons state is in the current move sequence, the program generates a new connection list
            #removing the persons state.  It then regenerates a new seq avoiding the person.
            loc=[p[0].location[0],p[0].location[1]]
            state=list(percepts['state_dict'].keys())[list(percepts['state_dict'].values()).index(loc)]
            
            #Removes person state from connections
            if agent.seq and state in agent.seq:
                for q,v in percepts['connections'].items():
                    if state in v:
                        del v[state]
                        
                #Resets agent sequence
                agent.seq =[]
                
                #Generates new sequence
                agent.seq = new_best_path=search(percepts['connections'], agent.state, agent.goal)
    
    #if the agent has a goal and it's not state0, check if the goal is reached, clean or continue search
    if agent.goal and agent.goal!='state0':
        if agent.state==agent.goal and agent.goal!='state0':
            for i in p0:
                if isinstance(i, Cleanable) and i.clean==False:
                    agent.goal=None
                    return 'clean'
        else:
            next_state=agent.seq.pop(0)

    #Else if we need a new goal, search for the next closest object.  Similar to last program
    elif agent.state==agent.goal or not agent.goal:
        for i in percepts['dirty_remaining']:
            j=list(percepts['state_dict'].keys())[list(percepts['state_dict'].values()).index(i)]
            if 'best_path' in locals():
                new_path=search(percepts['connections'], agent.state, j)
                if len(new_path)<len(best_path):
                    best_path=new_path.copy()
            else:
                best_path=search(percepts['connections'], agent.state, j)
                
        #If there are no dirty objects remaining, create a path back to state0
        if not percepts['dirty_remaining']:
            best_path=search(percepts['connections'], agent.state, 'state0')
        
        #Set the agent sequence
        agent.seq=best_path.copy()
        
        #Set the agent goal
        agent.goal=best_path[-1]
        
        next_state=agent.seq.pop(0)
    
    #skips the searching if we already have no new objects and continues home
    elif not percepts['dirty_remaining']:
        next_state=agent.seq.pop(0)

        
    
    next_move=percepts['state_dict'][next_state]
    agent.state=next_state
    if agent.location[0]+1==next_move[0]:
        return 'moveright'
    elif agent.location[0]-1 == next_move[0]:
        return 'moveleft'
    elif agent.location[1]+1== next_move[1]:
        return 'movedown'
    elif agent.location[1]-1==next_move[1]:
        return 'moveup'         
    

In [28]:
UniHub_3_3=Uni_Hub_Algorithm(7, {'COVIDRobot': (0,150,50), 'Trolley': (150,100,0), 'Chair': (100, 75, 0), 'Person': (250,0,0)})
agent=COVIDRobot(program)
UniHub_3_3.add_thing(agent, [0,0])
UniHub_3_3.run()
print("The COVIDRobot moved %d times and cleaned %d objects.  The overall performance (moves/objects) = %5.1f" % (agent.performance, agent.objects_cleaned, agent.performance/agent.objects_cleaned))

,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


The COVIDRobot moved 16 times and cleaned 2 objects.  The overall performance (moves/objects) =   8.0
